In [2]:
import numpy as np

In [3]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights with small random values to break symmetry.
        # (inputs, neurons) so we can dot product with inputs (samples, inputs)
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)

        # Initialize biases as zeros (one for each neuron)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        # Saving the input values for the backward pass calculations
        self.inputs = inputs

        # Calculate output: Dot product of inputs and weights, then add bias
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        """
        dvalues: The gradient of the loss with respect to the output of this layer.
                 Essentially, the "error signal" coming from the layer ahead.
        """
        # 1. Gradient on weights: how much did each weight affect the error?
        self.dweights = np.dot(self.inputs.T, dvalues)

        # 2. Gradient on biases: how much did each bias affect the error?
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        # 3. Gradient on inputs: error signal to pass to the PREVIOUS layer
        self.dinputs = np.dot(dvalues, self.weights.T)